In [162]:
from typing import List
import os
import re
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import requests

In [163]:
import os

from util.widgets import generate_ym, generate_ymd, generate_i

ym = generate_ym()
g = 'cargohose_herren'
bd = f'{g}/{ym}'
bd_a = f'{bd}/a'
b_d = f'{g}_{ym}'

os.makedirs(bd, exist_ok=True)
os.makedirs(bd_a, exist_ok=True)

ymd = generate_ymd()
i = '(08-09)'
bd_a_img = f'{bd_a}/{ymd}'
os.makedirs(bd_a_img, exist_ok=True)

In [164]:
erp_df: DataFrame = pd.read_excel(f'{bd_a}/product_20240724051948185.xls')

# 如果没有图片链接,说明已下架或者还没更新
erp_df.dropna(subset=['图片链接 1'], inplace=True)

erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,S3图片链接 8缩略图,S3图片链接 8,S3图片链接 9缩略图,S3图片链接 9,子图链接,长,宽,高,尺码表,印花名称
0,FBE230705004,NaN,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Woven Pocke...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Grey,Khaki,Dark Gray","M,L,XL,XXL,XXXL",NaN,...,NaN,https://lykjaiqgk57h5yqa.imgsapp.com/afsd/8867...,NaN,https://lg29bdziw7erh33a.imgsapp.com/afsd/0290...,NaN,30,20,3.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN
1,FBE230705004BKL,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 L,"<h1 align=""center""><span style=""font-size:14pt...",Black,L,Large,...,NaN,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN
2,FBE230705004BKL2,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXL,XX-Large,...,NaN,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN
3,FBE230705004BKL3,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXXL,3X-Large,...,NaN,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN
4,FBE230705004BKM,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 M,"<h1 align=""center""><span style=""font-size:14pt...",Black,M,Medium,...,NaN,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,ZUQ210805001KHXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 卡其色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XL,X-Large,...,NaN,https://hx1go8v2wwpz934l.imgsapp.com/afsd/5953...,NaN,https://h1j5vfmwyrvs0tcz.imgsapp.com/afsd/5953...,NaN,34,20,2.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN
300,ZUQ210805001WEL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 L 31,"<h1 align=""center""><span style=""font-size:14pt...",Wine,L,Large,...,NaN,https://8r5puatkpqu0zrez.imgsapp.com/afsd/5953...,NaN,https://q3ohinf7053vj4mk.imgsapp.com/afsd/5953...,NaN,40,35,1.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN
301,ZUQ210805001WEXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Wine,XL,X-Large,...,NaN,https://vt27v2ccgf0qmv4e.imgsapp.com/afsd/5953...,NaN,https://wzgtchmjc7fke2w2.imgsapp.com/afsd/5953...,NaN,29,20,4.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN
302,ZYN230315007,NaN,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Fashion Loose Large Cotton Shorts Men's ...,男士潮宽松大码棉短裤男多口袋休闲街头风七分工装裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Beige,Khaki,Brown,AG","29,30,31,32,33,34,36,A,B,C,D",NaN,...,NaN,https://ksuyxp84yskfbm6c.imgsapp.com/afsd/7969...,NaN,https://u0ccd73bi3lghs3c.imgsapp.com/afsd/7969...,NaN,25,25,2.0,Size: 29 Waist: 73cm/28.74'' Hip: 100cm/39.37'...,NaN


In [165]:
def generate_amazon_sku(s: str, c: str, n: str, b: int) -> str:
    """

    :param s: ERP的sku
    :param c: 仓库编号
    :param n: 节点
    :param b: 父sku批号
    :return:
    """
    p = re.compile(r'^(?P<p>[A-Z]+)(?P<d>\d{6})(?P<n>\d+)(?P<cs>.*?)$', re.S)
    # for i in p.match(s).groups():
    #     print(i)
    t = time.strftime('%Y-%m-%d', time.localtime())[2:]
    cs = p.match(s).group('cs')
    result = f'{c}-{n}-{t}-{b:03d}-{cs}'.strip('-')
    return result


s1 = 'BYJ240716009BWL2'
# MKT-DE-24-07-18-001-BWL2
generate_amazon_sku(s1, 'FRL', 'DE', 1)

'FRL-DE-24-07-26-001-BWL2'

In [166]:
erp_df['父SKU'] = erp_df['父SKU'].fillna(erp_df['SKU'])
erp_df['parent_child'] = np.where(erp_df['父SKU'] == erp_df['SKU'], 'Parent', 'Child')

archived_skus = open('./util/skus/archived_skus.txt', mode='r', encoding='utf-8').readlines()
archived_skus = [i.strip() for i in archived_skus]
other_skus = open('./util/skus/other_skus.txt', mode='r', encoding='utf-8').readlines()
other_skus = [i.strip() for i in other_skus]

erp_df = erp_df[~erp_df['父SKU'].isin(archived_skus + other_skus)]

erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,S3图片链接 8,S3图片链接 9缩略图,S3图片链接 9,子图链接,长,宽,高,尺码表,印花名称,parent_child
0,FBE230705004,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Woven Pocke...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Grey,Khaki,Dark Gray","M,L,XL,XXL,XXXL",NaN,...,https://lykjaiqgk57h5yqa.imgsapp.com/afsd/8867...,NaN,https://lg29bdziw7erh33a.imgsapp.com/afsd/0290...,NaN,30,20,3.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Parent
1,FBE230705004BKL,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 L,"<h1 align=""center""><span style=""font-size:14pt...",Black,L,Large,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
2,FBE230705004BKL2,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXL,XX-Large,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
3,FBE230705004BKL3,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXXL,3X-Large,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
4,FBE230705004BKM,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 M,"<h1 align=""center""><span style=""font-size:14pt...",Black,M,Medium,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,ZUQ210805001KHXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 卡其色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XL,X-Large,...,https://hx1go8v2wwpz934l.imgsapp.com/afsd/5953...,NaN,https://h1j5vfmwyrvs0tcz.imgsapp.com/afsd/5953...,NaN,34,20,2.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN,Child
300,ZUQ210805001WEL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 L 31,"<h1 align=""center""><span style=""font-size:14pt...",Wine,L,Large,...,https://8r5puatkpqu0zrez.imgsapp.com/afsd/5953...,NaN,https://q3ohinf7053vj4mk.imgsapp.com/afsd/5953...,NaN,40,35,1.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN,Child
301,ZUQ210805001WEXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Wine,XL,X-Large,...,https://vt27v2ccgf0qmv4e.imgsapp.com/afsd/5953...,NaN,https://wzgtchmjc7fke2w2.imgsapp.com/afsd/5953...,NaN,29,20,4.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN,Child
302,ZYN230315007,ZYN230315007,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Fashion Loose Large Cotton Shorts Men's ...,男士潮宽松大码棉短裤男多口袋休闲街头风七分工装裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Beige,Khaki,Brown,AG","29,30,31,32,33,34,36,A,B,C,D",NaN,...,https://ksuyxp84yskfbm6c.imgsapp.com/afsd/7969...,NaN,https://u0ccd73bi3lghs3c.imgsapp.com/afsd/7969...,NaN,25,25,2.0,Size: 29 Waist: 73cm/28.74'' Hip: 100cm/39.37'...,NaN,Parent


In [167]:
erp_df['父SKU'].unique()

array(['FBE230705004', 'FZQ240108001', 'HRR210316820', 'HYM230921003',
       'LBB230821017', 'LJL240311003', 'MYL240709002', 'QXP240706007',
       'SQI220215172', 'SQI221104003', 'ZHS90612722', 'ZUQ210805001',
       'ZYN230315007'], dtype=object)

In [168]:
s1 = [f'图片链接 {i}' for i in range(1, 10)]
s2 = [f'S3图片链接 {i}' for i in range(1, 10)]

src_df = pd.concat([erp_df['父SKU'], erp_df[s1], erp_df[s2]], axis=1)
src_df

,父SKU,图片链接 1,图片链接 2,图片链接 3,图片链接 4,图片链接 5,图片链接 6,图片链接 7,图片链接 8,图片链接 9,S3图片链接 1,S3图片链接 2,S3图片链接 3,S3图片链接 4,S3图片链接 5,S3图片链接 6,S3图片链接 7,S3图片链接 8,S3图片链接 9
0,FBE230705004,http://www.welenespicshop.com/tfkmg5/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg5/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,https://0pskaph6ux75hs2f.imgsapp.com/afsd/0290...,https://wnuw0l6qtra55la3.imgsapp.com/afsd/8867...,https://ljur1adde1ygi9uy.imgsapp.com/afsd/8867...,https://gsw2n0xlhdq7gr2v.imgsapp.com/afsd/8867...,https://n0odient09wy29g4.imgsapp.com/afsd/8867...,https://t3289gvjzou0wdf6.imgsapp.com/afsd/8867...,https://ukza98ci67f4ng0a.imgsapp.com/afsd/8867...,https://lykjaiqgk57h5yqa.imgsapp.com/afsd/8867...,https://lg29bdziw7erh33a.imgsapp.com/afsd/0290...
1,FBE230705004,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,NaN,NaN,NaN,NaN,https://7g4ekjo4hy8cunoy.imgsapp.com/afsd/8867...,https://j88ygb2vsvregeby.imgsapp.com/afsd/8867...,https://2m1qlzvv31ya0v8h.imgsapp.com/afsd/8867...,https://y4npn7vhqmr0qrf7.imgsapp.com/afsd/8867...,https://4qb8319ym7j28btn.imgsapp.com/afsd/8867...,NaN,NaN,NaN,NaN
2,FBE230705004,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,NaN,NaN,NaN,NaN,https://gwwbg4i3shjqcntl.imgsapp.com/afsd/8867...,https://kkbx83onk6tj9rfl.imgsapp.com/afsd/8867...,https://j7wo61eime7h4upk.imgsapp.com/afsd/8867...,https://gqh17mrxvy30pmzu.imgsapp.com/afsd/8867...,https://kt7xekbbf5vxglq3.imgsapp.com/afsd/8867...,NaN,NaN,NaN,NaN
3,FBE230705004,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,NaN,NaN,NaN,NaN,https://lf8munj703juy0st.imgsapp.com/afsd/8867...,https://uehvtongl40wu7vn.imgsapp.com/afsd/8867...,https://yenpqa1ap7m3doz4.imgsapp.com/afsd/8867...,https://5e2iougmhd3bpftg.imgsapp.com/afsd/8867...,https://jofxvjto5er0310l.imgsapp.com/afsd/8867...,NaN,NaN,NaN,NaN
4,FBE230705004,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,http://www.welenespicshop.com/tfkmg3/FBE230705...,NaN,NaN,NaN,NaN,https://cp8yyoqz1xg4u7c1.imgsapp.com/afsd/8867...,https://q9ijv13euxw1qj6x.imgsapp.com/afsd/8867...,https://7f7ikbr9dvs6g34w.imgsapp.com/afsd/8867...,https://tzwq8jirb8ll8bgw.imgsapp.com/afsd/8867...,https://y8vwh0evtu12gjpc.imgsapp.com/afsd/8867...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,ZUQ210805001,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,http://www.welenespicshop.com/tupian13/ZUQ2108...,https://2qjcrnhs21cjbvqf.imgsapp.com/afsd/5953...,https://jjeqqorfv1nd8ej4.imgsapp.com/afsd/5953...,https://z49v4roez86gw80d.imgsapp.com/afsd/5953...,https://sapre39bl9xosxiz.imgsapp.com/afsd/5953...,

In [169]:
src_unique_df = pd.DataFrame()

for i in range(1, 1 + 9):
    src_unique_df = pd.concat([src_unique_df, pd.DataFrame(src_df[f'图片链接 {i}'].unique())])

src_unique_df.dropna(subset=[0], inplace=True)
src_unique_df.drop_duplicates(subset=[0], inplace=True)
src_unique_df['fn'] = src_unique_df[0].apply(lambda i: i.rsplit('/', 1)[-1])
src_unique_df

,0,fn
0,http://www.welenespicshop.com/tfkmg5/FBE230705...,FBE230705004_0155100506215803694.jpg
1,http://www.welenespicshop.com/tfkmg3/FBE230705...,FBE230705004_0447369901376482770.jpg
2,http://www.welenespicshop.com/tfkmg3/FBE230705...,FBE230705004_0448147531338124763.jpg
3,http://www.welenespicshop.com/tfkmg3/FBE230705...,FBE230705004_0447514311316755720.jpg
4,http://mp3.ser.ltd:28099/image/product/ftp/tfk...,FBE230705004_0154444026212899385.jpg
...,...,...
22,http://www.welenespicshop.com/tupian13/ZUQ2108...,"ZUQ210805001DYM,ZUQ210805001DYL,ZUQ210805001DY..."
23,http://www.welenespicshop.com/tupian13/ZUQ2108...,"ZUQ210805001KHM,ZUQ210805001KHL,ZUQ210805001KH..."
24,http://www.welenespicshop.com/tupian13/ZUQ2108...,"ZUQ210805001WEM,ZUQ210805001WEL,ZUQ210805001WE..."
25,http://www.welenespicshop.com/tfkmg1/ZYN230315...,ZYN230315007_0437233312010786277.jpg


In [170]:
len(src_unique_df['fn'].unique())

359

In [176]:
img_home = f'https://www.gitee.com/wuku32/project-step-2/{bd_a_img}/'


def redirect(u: str):
    u = re.sub(r'http://.*/', img_home, u)
    return u


s = ''
redirect(s)

''

In [172]:
erp_df

,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,S3图片链接 8,S3图片链接 9缩略图,S3图片链接 9,子图链接,长,宽,高,尺码表,印花名称,parent_child
0,FBE230705004,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Woven Pocke...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Grey,Khaki,Dark Gray","M,L,XL,XXL,XXXL",NaN,...,https://lykjaiqgk57h5yqa.imgsapp.com/afsd/8867...,NaN,https://lg29bdziw7erh33a.imgsapp.com/afsd/0290...,NaN,30,20,3.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Parent
1,FBE230705004BKL,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 L,"<h1 align=""center""><span style=""font-size:14pt...",Black,L,Large,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
2,FBE230705004BKL2,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXL,XX-Large,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
3,FBE230705004BKL3,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXXL,3X-Large,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
4,FBE230705004BKM,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 M,"<h1 align=""center""><span style=""font-size:14pt...",Black,M,Medium,...,NaN,NaN,NaN,NaN,34,22,1.0,Size: M Waist: 72-82cm/28.35-32.28'' Hip: 108c...,NaN,Child
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,ZUQ210805001KHXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 卡其色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XL,X-Large,...,https://hx1go8v2wwpz934l.imgsapp.com/afsd/5953...,NaN,https://h1j5vfmwyrvs0tcz.imgsapp.com/afsd/5953...,NaN,34,20,2.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN,Child
300,ZUQ210805001WEL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 L 31,"<h1 align=""center""><span style=""font-size:14pt...",Wine,L,Large,...,https://8r5puatkpqu0zrez.imgsapp.com/afsd/5953...,NaN,https://q3ohinf7053vj4mk.imgsapp.com/afsd/5953...,NaN,40,35,1.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN,Child
301,ZUQ210805001WEXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Wine,XL,X-Large,...,https://vt27v2ccgf0qmv4e.imgsapp.com/afsd/5953...,NaN,https://wzgtchmjc7fke2w2.imgsapp.com/afsd/5953...,NaN,29,20,4.0,Size: M Waist: 78cm/30.71'' Hip: 94cm/37.01'' ...,NaN,Child
302,ZYN230315007,ZYN230315007,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Fashion Loose Large Cotton Shorts Men's ...,男士潮宽松大码棉短裤男多口袋休闲街头风七分工装裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Beige,Khaki,Brown,AG","29,30,31,32,33,34,36,A,B,C,D",NaN,...,https://ksuyxp84yskfbm6c.imgsapp.com/afsd/7969...,NaN,https://u0ccd73bi3lghs3c.imgsapp.com/afsd/7969...,NaN,25,25,2.0,Size: 29 Waist: 73cm/28.74'' Hip: 100cm/39.37'...,NaN,Parent


In [178]:
for _ in s1:
    erp_df[_] = erp_df[_].astype(str)
    erp_df[f'gitee{_}'] = erp_df[_].apply(lambda i: redirect(i))

erp_df

C:\Users\000\AppData\Local\Temp\ipykernel_12932\1606739646.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erp_df[_] = erp_df[_].astype(str)
C:\Users\000\AppData\Local\Temp\ipykernel_12932\1606739646.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  erp_df[f'gitee{_}'] = erp_df[_].apply(lambda i: redirect(i))


,SKU,父SKU,产品缩略图,产品类型,标题,产品名称,描述,颜色,尺寸,亚马逊尺寸,...,parent_child,gitee图片链接 1,gitee图片链接 2,gitee图片链接 3,gitee图片链接 4,gitee图片链接 5,gitee图片链接 6,gitee图片链接 7,gitee图片链接 8,gitee图片链接 9
0,FBE230705004,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Woven Pocke...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤,"<h1 align=""center""><span style=""font-size:14pt...","Black,Grey,Khaki,Dark Gray","M,L,XL,XXL,XXXL",NaN,...,Parent,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...
1,FBE230705004BKL,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 L,"<h1 align=""center""><span style=""font-size:14pt...",Black,L,Large,...,Child,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,nan,nan,nan,nan
2,FBE230705004BKL2,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXL,XX-Large,...,Child,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,nan,nan,nan,nan
3,FBE230705004BKL3,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 XXXL,"<h1 align=""center""><span style=""font-size:14pt...",Black,XXXL,3X-Large,...,Child,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,nan,nan,nan,nan
4,FBE230705004BKM,FBE230705004,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Mens Casual Waist Color Sports Hat Multi Woven...,男士四季街头休闲运动多口袋脚口帽绳腰间系带纯色工装长裤卫裤 黑色 M,"<h1 align=""center""><span style=""font-size:14pt...",Black,M,Medium,...,Child,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,ZUQ210805001KHXL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 卡其色 XL 32,"<h1 align=""center""><span style=""font-size:14pt...",Khaki,XL,X-Large,...,Child,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...,https://www.gitee.com/wuku32/project-step-2/ca...
300,ZUQ210805001WEL,ZUQ210805001,NaN,普通产品 -> Clothing Shoes & Accessories -> 男装 -> ...,Men's Thin Trousers Solid Color High Waist Ela...,男士薄款纯色高腰弹力休闲商务西装长裤 酒红色 L 31,"<h1 align=""center""><span style=""font-size:14pt...",Wine,L,Large,...